In [ ]:

table=['OPEN_BIO', 'OPEN_MEDIC', 'OPEN_DAMIR', 'OPEN_LPP']
lib=["OPEN BIO (base complète sur les dépenses de biologie médicale interrégimes)","OPEN MEDIC (base complète sur les dépenses de médicaments)","OPEN DAMIR (base complète sur les dépenses d'assurance maladie Inter-Régimes)","OPEN LPP (base complète sur les dépenses de dispositifs médicaux inscrits à la liste des produits et prestations (LPP))"]
tableexist={
    'tables':table,
    'tab_lib':lib,
    'produit':"OPEN_DATA"
}

In [ ]:
pip install tableschema

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 2.0 MB/s 
     |████████████████████████████████| 72 kB 289 kB/s 
     |████████████████████████████████| 41 kB 368 kB/s 
     |████████████████████████████████| 133 kB 70.0 MB/s 
     |████████████████████████████████| 132 kB 72.7 MB/s 
     |████████████████████████████████| 10.2 MB 73.2 MB/s 
     |████████████████████████████████| 79 kB 6.0 MB/s 
     |████████████████████████████████| 127 kB 74.8 MB/s 
  Created wheel for linear-tsv: filename=linear_tsv-1.1.0-py3-none-any.whl size=7401 sha256=ff2e8f6803bf6801ca21b3057c6a3edb58175fcae97c2b609d1182b0d94d3685
  Stored in directory: /root/.cache/pip/wheels/31/25/a5/d9b4b59636e1b9f183b7e1022e7be5c51141717b92398d7f9e
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10766 sha256=3bfcf8a071ed2fd253913d014d39e8b42f51731a582e81e18ae1618d3e1230b6
  Stored in directory

In [ ]:
from tableschema import Schema
import json
for table in tableexist['tables']:
  with open('{}.json'.format(table),'w', encoding='utf-8') as f:
    json.dump([],f, indent=4,ensure_ascii=False)
    import pandas as pd 
    df= pd.read_csv("{}.csv".format(table), sep=",",encoding='utf-8',na_values="")
    df.rename(columns={'Variable':'variables','Libellé':'lib'}, inplace=True)
    df=df.fillna('')


    ##### transformation des types 
    mapping_t = {
      "char": "string",
      "num":"integer",
      "date":"date"
    }
    df["type"]=df["Type"].map(mapping_t)
    # Fonction qui permet de classer le type en number ou integer en fct de la taille 
    def numbint(x): 
      res = re.search(",", x)
      if res:
        return "number"
      return "integer"
    df["datecreated"]=''
    df["datedeleted"]=''
    df["datemissing"]=''
    df['observation']=''
    df['nomenclature']=''
    df["rdg"]=''
    df["length"]=''
    df['table']=table


  # Creation dy type oracle 
    mapping_to = {
      "string": "string",
      "date":"date",
      "integer":"number"
      
      }

    df["type_oracle"]=df["type"].map(mapping_to)

 
# Creation des fichiers json 
  for i in range(len(df['variables'])):
    if df['type'][i]=='number':
      df['type'][i]=df['length'][i].apply(numbint)    
     # les colonnes manquante  
  df=df.fillna('')
  
# create json file with the tables name

  for z in range(len(df['variables'])): 
    new_dict={
            "name": df["variables"][z],
            "description": df["lib"][z],
            "type": df["type"][z],
            "nomenclature": df["nomenclature"][z],
            "length": str(df["length"][z]),
            "format": "default",
            "dateCreated": str(df["datecreated"][z]),
            "dateDeleted": str(df["datedeleted"][z]),
            "dateMissing": df["datemissing"][z],
            "observation": df["observation"][z],
            "regle_gestion": df["rdg"][z],
            "type_oracle": df["type_oracle"][z]
       
        }
    
    with open('{}.json'.format(table),encoding='utf-8') as f:  
      data=json.load(f)
      data.append(new_dict)
    with open('{}.json'.format(table), 'w', encoding='utf-8') as f:
      json.dump(data, f, indent=4, ensure_ascii=False)
  # Mettons les fichiers sous format tableschema

from tableschema import Schema
for table in tableexist['tables']:
  with open('{}.json'.format(table), "r", encoding='utf-8') as f:
    descriptor = {'fields': json.load(f)}
    schema=Schema(descriptor) 
    schema.descriptor['name']=table
    schema.descriptor['title']=tableexist['tab_lib'][tableexist['tables'].index(table)]
    schema.descriptor['produit']=tableexist['produit']
    schema.descriptor['missingValues'] = ['']
    if table=='OPEN_DAMIR':
      schema.descriptor['history']={"dateCreated": "2009",
                                    "dateDeleted": "",
                                    "dateMissing": []}
    else:
      schema.descriptor['history']={"dateCreated": "2014",
                                    "dateDeleted": "",
                                 "dateMissing": []} 

      schema.descriptor['champ']=""
      schema.descriptor['observation']=""                              
    schema.commit()
    schema.save('{}.json'.format(table),ensure_ascii=False)